# Step 2: ANALYZE

In [28]:
import pandas as pd
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
import re
import pandas as pd
import numpy as np
import platform
import matplotlib.pyplot as plt
import json


In [29]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as SW_en
from spacy.lang.es.stop_words import STOP_WORDS as SW_es

In [30]:
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

In [31]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [32]:
#import nltk 
#from nltk.corpus import stopwords 
#set(stopwords.words('english'))


In [33]:
#!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_sm
#!python -m spacy download es_core_news_md

In [34]:
import webbrowser

In [35]:
##################################333 PARAMETERS #####################

key_words_DA = ['data analyst', 'analisis de datos',
                      'data analytics', 'analitica de datos',
                      'business analyst', 'analisis de negocios',
                      'sql',
                      'python',
                      'business intelligence', 'inteligencia de negocios', ' bi ',
                      'excel', 'microsoft office', 'ms office',
                      'dashboard', 'kpi',
                      'reporting', 'informes',
                      'tableau', 'powerbi', 'power bi', 'microsoft bi', 'qlik', 'spotfire',
                      ]

key_words_DS = ['data analyst', 'analisis de datos',
                      'data analytics', 'analitica de datos',
                      'business analyst', 'analisis de negocios',
                      'sql',
                      'python',
                      'business intelligence', 'inteligencia de negocios', ' bi ',
                      'excel', 'microsoft office', 'ms office',
                      'dashboard', 'kpi',
                      'reporting', 'informes',
                      'tableau', 'powerbi', 'power bi', 'microsoft bi', 'qlik', 'spotfire',
                      ]

key_words_DE = ['data engineer', 'ingeniero de datos',
                       'sql', 'nosql', 'no sql', 'bases de datos no relacionales',
                       'python', ' r ', 'scala', 'java ', 'spark', 'hadoop', 'haddop' 'hive', 'impala', 'kafka',
                       'data wrangling', 'limpieza de datos',
                       'etl', 'extract, transform and load', 'map reduce', 'mapreduce',
                       'data acquisition', 'adquisicion de datos',
                       'big data', 'bigdata', 'bigquery',
                       'data warehouse',
                       ' aws ', 'azure',
                       'software engineering'
                       ]

dic_key_words = \
    {'data analyst'    : key_words_DA,
     'data scientist'  : key_words_DS,
     'data engineer'   : key_words_DE,
     'analista datos'  : key_words_DA,
     'cientifico datos': key_words_DS,
     'ingeniero datos' : key_words_DE
     }

In [36]:
def job_key_generator(job):
    job_key_s = ""
    job_name_splitted = job.split(" ")
    for string in job_name_splitted:
        job_key_s += string[0].upper()
        #job_key_s = job_key_s
    
    return job_key_s

## Getting csv

#### > PROFILE:

In [37]:
platform.system()

'Linux'

In [38]:
if platform.system() == 'Linux':
    df_profile = pd.read_csv(f'../data/raw/df_profile/df_profile_cristopherrojaslepe_2020-04-23 17.29.csv',
                             sep = ';',
                             encoding= 'utf8',
                             index_col = 0
                              )
    
elif platform.system() == 'Windows':
    df_profile = pd.read_csv(f'..\\data\\raw\\df_profile.csv',
                             sep = ';',
                             encoding= 'utf8'
                            )
    
df_profile

,info
profile,cristopherrojaslepe
headline,data scientist & analyst | python | pandas | n...
open new jobs,YES
about,data scientist & analyst and industrial engine...
experiences,"['collection coordination analyst', 'real esta..."
years of experiences,"['1 yr 2 mos', '1 yr', '2 yrs 1 mo', '8 mos', ..."
total years,7
degrees,"['master', 'engineering - be']"
fields,"['data analytics', 'industrial engineering']"
skills,"['analytical skills', 'bash', 'business analys..."


In [39]:
## ONLY FOR JUPYTER !!!!!!!!!!
# df_profile.rename(columns={'Unnamed: 0':'type'},inplace=True)
# df_profile.set_index('type', inplace=True)

#### > JOBS:

In [42]:
#f'../data/raw/df_jobs_Madrid_2020-04-18 23.50.csv',
#f'..\\data\\raw\\df_jobs_Madrid_2020-04-18 23.50.csv'
if platform.system() == 'Linux':
    df_jobs_raw = pd.read_csv(f'../data/raw/df_jobs/df_jobs_Barcelona_es_2020-04-23 00.40_100.csv',
                              sep = ';',
                              encoding= 'utf8',
                  )
elif platform.system() == 'Windows':
    df_jobs_raw = pd.read_csv(f'..\\data\\raw\\df_jobs\\df_jobs_Madrid_2020-04-20 01.38.csv',
                              sep = ';',
                              encoding= 'utf8',
                  )

In [43]:
df_jobs = df_jobs_raw

In [44]:
df_jobs.shape

(100, 18)

In [45]:
df_jobs.head(3)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Job Description,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info
0,analista datos,Barcelona,2020-04-23 00:40,1712815780,https://www.linkedin.com/jobs/view/1712815780/,Analista Liferay,everis,"Barcelona y alrededores, España",1 week,2020-04-16 00:40,True,¿Buscas un reto profesional que te saque de tu...,"{'yes': ['Foreign Languages', 'Decision-Making...",Associate,Program Development,Full-time,Consulting,Posted Date\nPosted 1 week ago\nNumber of view...
1,analista datos,Barcelona,2020-04-23 00:40,1813681738,https://www.linkedin.com/jobs/view/1813681738/,"Analista, programadores/as y manager Salesforce",Experis España,"Barcelona, Catalonia, Spain",2 weeks,2020-04-09 00:40,True,Experis selecciona diferentes perfiles con exp...,"{'yes': ['Business', 'Business Analysis'], 'no...",Mid-Senior level,Information Technology & Services,Full-time,Information Technology,Posted Date\nPosted 2 weeks ago\nNumber of vie...
2,analista datos,Barcelona,2020-04-23 00:40,1824218680,https://www.linkedin.com/jobs/view/1824218680/,WMS Analyst,Randstad España,"Barcelona, Cataluña / Catalunya, España",6 days,2020-04-17 00:40,True,Our client is based in Palau de Plegamans (30 ...,"{'yes': ['Business', 'Analytical Skills', 'Cus...",Mid-Senior level,Logistics & Supply Chain Food & Beverages,Full-time,Information Technology,Posted Date\nPosted 6 days ago\nNumber of view...


In [46]:
# ONLY IN CASE TO CORRECT WRONG DF
#df_jobs.drop(columns='Job htmlJob name', inplace=True)

#df_jobs = df_jobs[['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
#       'Job html', 'Job name', 'Company name', 'Company location',
#       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
#       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
#       'Job info']]

In [47]:
df_jobs[df_jobs['JOB TITLE']=='data analyst'].shape

(0, 18)

In [48]:
df_jobs[df_jobs['JOB TITLE']=='data scientist'].shape

(0, 18)

In [49]:
df_jobs[df_jobs['JOB TITLE']=='data engineer'].shape

(0, 18)

## Analizyng info

In [50]:
location = df_jobs.loc[0,'LOCATION']
location

'Barcelona'

### > Cleaning jobs:

In [51]:
df_jobs.loc[0,'JOB TITLE':'Current Job Id']

JOB TITLE            analista datos
LOCATION                  Barcelona
SEARCH DATETIME    2020-04-23 00:40
Current Job Id           1712815780
Name: 0, dtype: object

##### * JOB TITLE_ID

In [52]:
df_jobs['JOB TITLE_ID'] = df_jobs.apply(lambda x: x['JOB TITLE'] +"_"+ str(x['Current Job Id']), axis = 1 )

In [53]:
df_jobs.shape[0] 

100

In [54]:
len(df_jobs['JOB TITLE_ID'].unique())

100

In [55]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['JOB TITLE_ID'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

,LOCATION
JOB TITLE_ID,
analista datos_1560384265,1
analista datos_1831986272,1
cientifico datos_1812112306,1
cientifico datos_1812112304,1
cientifico datos_1809134908,1
...,...
analista datos_1781618413,1
analista datos_1781372547,1
analista datos_1780186114,1


In [56]:
df_jobs[df_jobs['Current Job Id']==1774326710]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Job Description,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID


In [57]:
df_jobs.shape[0] - len(df_jobs['JOB TITLE_ID'].unique())

0

In [58]:
df_jobs.drop_duplicates(subset='JOB TITLE_ID', keep='first', inplace = True)

In [59]:
df_jobs.shape

(100, 19)

##### * Current Job Id

In [60]:
len(df_jobs['Current Job Id'].unique())

100

In [61]:
pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['Current Job Id'],
               aggfunc=np.count_nonzero        
              ).sort_values(by=['LOCATION'], ascending=False)

,LOCATION
Current Job Id,
1422096341,1
1812136544,1
1820264957,1
1820264742,1
1820246077,1
...,...
1771493337,1
1766881858,1
1750652346,1


In [62]:
unique_jobs = list(df_jobs['JOB TITLE'].unique())
unique_jobs

['analista datos', 'cientifico datos', 'ingeniero datos']

In [63]:
pivot_ID_TITLES = pd.pivot_table(df_jobs,
               values="LOCATION",
               index=['Current Job Id'],
               columns=['JOB TITLE'],
               aggfunc=np.count_nonzero        
              ).reset_index()

pivot_ID_TITLES['Multiple Job Post'] = ""

for j in unique_jobs:
    pivot_ID_TITLES[j] = \
    pivot_ID_TITLES[j].apply(lambda x: job_key_generator(j) if x==1.0 else '')
    pivot_ID_TITLES['Multiple Job Post'] = pivot_ID_TITLES['Multiple Job Post'] + pivot_ID_TITLES[j] + '-'

pivot_ID_TITLES

JOB TITLE,Current Job Id,analista datos,cientifico datos,ingeniero datos,Multiple Job Post
0,1422096341,,,ID,--ID-
1,1516530143,,CD,,-CD--
2,1524810490,,CD,,-CD--
3,1560384265,AD,,,AD---
4,1598454977,,,ID,--ID-
...,...,...,...,...,...
95,1832936591,,,ID,--ID-
96,1832936642,,,ID,--ID-
97,1833195239,,CD,,-CD--
98,1833195441,AD,,,AD---


In [64]:
df_jobs = df_jobs.merge(pivot_ID_TITLES[['Current Job Id','Multiple Job Post']],
                       on='Current Job Id',
                       how='left')

In [65]:
df_jobs[df_jobs['Current Job Id']==1645918674]

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Job Description,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID,Multiple Job Post


In [66]:
df_jobs.drop_duplicates(subset='Current Job Id', keep='first')

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Job Description,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info,JOB TITLE_ID,Multiple Job Post
0,analista datos,Barcelona,2020-04-23 00:40,1712815780,https://www.linkedin.com/jobs/view/1712815780/,Analista Liferay,everis,"Barcelona y alrededores, España",1 week,2020-04-16 00:40,True,¿Buscas un reto profesional que te saque de tu...,"{'yes': ['Foreign Languages', 'Decision-Making...",Associate,Program Development,Full-time,Consulting,Posted Date\nPosted 1 week ago\nNumber of view...,analista datos_1712815780,AD---
1,analista datos,Barcelona,2020-04-23 00:40,1813681738,https://www.linkedin.com/jobs/view/1813681738/,"Analista, programadores/as y manager Salesforce",Experis España,"Barcelona, Catalonia, Spain",2 weeks,2020-04-09 00:40,True,Experis selecciona diferentes perfiles con exp...,"{'yes': ['Business', 'Business Analysis'], 'no...",Mid-Senior level,Information Technology & Services,Full-time,Information Technology,Posted Date\nPosted 2 weeks ago\nNumber of vie...,analista datos_1813681738,AD---
2,analista datos,Barcelona,2020-04-23 00:40,1824218680,https://www.linkedin.com/jobs/view/1824218680/,WMS Analyst,Randstad España,"Barcelona, Cataluña / Catalunya, España",6 days,2020-04-17 00:40,True,Our client is based in Palau de Plegamans (30 ...,"{'yes': ['Business', 'Analytical Skills', 'Cus...",Mid-Senior level,Logistics & Supply Chain Food & Beverages,Full-time,Information Technology,Posted Date\nPosted 6 days ago\nNumber of view...,analista datos_1824218680,AD---
3,analista datos,Barcelona,2020-04-23 00:40,1801660398,https://www.linkedin.com/jobs/view/1801660398/,Analista de datos,Banco Sabadell,"San Cugat del Vallés, Cataluña / Catalunya, Es...",3 weeks,2020-04-02 00:40,True,"Misión\nObtener, depurar y presentar informaci...","{'yes': ['SQL', 'Analytical Skills', 'Computer...",Associate,Banking,Full-time,Analyst Engineering Information Technology,Posted Date\nPosted 3 weeks ago\nNumber of vie...,analista datos_1801660398,AD---
4,analista datos,Barcelona,2020-04-23 00:40,1779999677,https://www.linkedin.com/jobs/view/1779999677/,Analista de Datos,Outlet de Viviendas,"Barcelona, ES",1 month,2020-03-26 00:40,False,"Company Description: OUTLET DE VIVIENDAS, empr...",{},Entry level,Commercial Real Estate Human Resources Real Es...,Full-time,Information Technology,Posted Date\nPosted 1 month ago\nNumber of vie...,analista datos_1779999677,AD---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ingeniero datos,Barcelona,2020-04-23 00:40,1832936591,https://www.linkedin.com/jobs/view/1832936591/,Data Scientist Internship,Entravision,"Barcelona, ES",3 days,2020-04-20 00:40,False,At the heart of our technology are our Machine...,{},Internship,Marketing & Advertising Online Media Internet,Internship,Engineering Information Technology,Posted Date\nPosted 3 days ago\nNumber of view...,ingeniero datos_1832936591,--ID-
96,ingeniero datos,Barcelona,2020-04-23 00:40,1772485017,https://www.linkedin.com/jobs/view/1772485017/,Senior Big Data Engineer,Claire Joster,"Barcelona, ES",1 month,2020-03-26 00:40,False,Job title:Senior Big Data Engineer\nJob type:P...,{},Associate,Information Technology & Services Internet Hum...,Full-time,Engineering Information Technology,Posted Date\nPosted 1 month ago\nNumber of vie...,ingeniero datos_1772485017,--ID-
97,ingeniero datos,Barcelona,2020-04-23 00:40,1832911914,https://www.linkedin.com/jobs/view/1832911914/,Senior Data Engineer,Stuart,"Barcelona, ES",1 day,2020-04-22 00:40,False,SVGs not supported by this browser. Facebook S...,{},Associate,Information Technology & Services Computer Sof...,Full-time,Information Technology,Posted Date\nPosted 1 day ago\nNumber of views...,ingeniero datos_1832911914,--ID-
98,ingeniero datos,Barcelona,2020-04-23 00:40,1832912351,https://www.linkedin.com/jobs/view/18329

### > Language of each job description:

In [67]:
df_jobs['Job name'].head(10)

0                                     Analista Liferay
1      Analista, programadores/as y manager Salesforce
2                                          WMS Analyst
3                                    Analista de datos
4                                    Analista de Datos
5                        Innova-tsn: Analista de datos
6                                  Senior Data Analyst
7                                Analista de Postventa
8    Snr Data Analyst – PepsiCo Global Template - I...
9                       Analista de Datos (Matemático)
Name: Job name, dtype: object

In [68]:
detect(df_jobs.loc[6,'Job Description'])

'en'

In [69]:
#New comñumn identifying language
df_jobs['Info Language'] = df_jobs['Job Description'].apply(lambda x: detect(x))

In [70]:
df_jobs['Info Language'].unique()

array(['es', 'en', 'ca'], dtype=object)

In [71]:
df_jobs['Seniority Level'].unique()

array(['Associate', 'Mid-Senior level', 'Entry level', nan, 'Executive',
       'Internship'], dtype=object)

### > Coincidences with definitions:

In [72]:
def coincidencias(job_skills, post_skills):
    c = 0
    for i in job_skills: 
        if re.search(i,post_skills): 
            c+=1
    return c
        

In [73]:
def list_coincidencias(job_skills, post_skills):
    l=[]
    for i in job_skills:
        if re.search(i,post_skills):
            l.append(i)
    return l
        

In [74]:
def clean_text(text):
    text = text.replace('\n',' ')
    text = text.replace('á','a')
    text = text.replace('é','e')
    text = text.replace('í','i')
    text = text.replace('ó','o')
    text = text.replace('ú','u')
    text = text.replace('.',' ')
    text = text.lower()
    
    return text
    

#### jobs >

In [75]:
#jobs_list = list(df_jobs['JOB TITLE'].unique()) #only jobs that searcht has been taken
unique_jobs = list(df_jobs['JOB TITLE'].unique())

In [76]:
#New columns with coindicences DEFINITION vs Job post
coincidences_jobs = []
for j in unique_jobs:
    skills_list = dic_key_words.get(j) #dictionary
    coincidences_jobs.append(f'{j[0:2]} coincidences')
    df_jobs[f'{j[0:2]} coincidences'] = \
    df_jobs['Job Description'].apply(lambda x: coincidencias(skills_list,clean_text(x)))
    df_jobs[f'{j[0:2]} coincidences_list'] = \
    df_jobs['Job Description'].apply(lambda x: list_coincidencias(skills_list,clean_text(x)))

#     df_jobs['DS coincidences'] = \
#     df_jobs['Job info'].apply(lambda x: coincidencias(DS_list,clean_text(x)))
#     df_jobs['DS coincidences_list'] = \
#     df_jobs['Job info'].apply(lambda x: list_coincidencias(DS_list,clean_text(x)))

#     df_jobs['DE coincidences'] = \
#     df_jobs['Job info'].apply(lambda x: coincidencias(DE_list,clean_text(x)))
#     df_jobs['DE coincidences_list'] = \
#     df_jobs['Job info'].apply(lambda x: list_coincidencias(DE_list,clean_text(x)))

#### profile >

In [77]:
df_profile.loc['total skills', 'info']

"['analytical skills', 'bash', 'business analysis', 'business intelligence', 'business process improvement', 'continuous improvement', 'data analysis', 'data analytics', 'data modeling', 'data pipelines', 'data science', 'data visualization', 'engineering', 'engineering - be', 'english', 'español', 'financial analysis', 'git', 'industrial engineering', 'industry knowledge', 'jupyter', 'leadership', 'lean manufacturing', 'machine learning', 'master', 'microsoft excel', 'microsoft office', 'microsoft power bi', 'microsoft power query', 'microsoft project', 'mysql', 'negotiation', 'numpy', 'pandas', 'power pivot', 'process engineering', 'process optimization', 'programming', 'project management', 'python', 'r', 'sap', 'scrum', 'seaborn', 'six sigma', 'spanish', 'sql', 'statistics', 'tableau', 'team management', 'teamwork', 'telco', 'vba', 'visual basic for applications', 'web scraping']"

In [78]:
profile_skills = clean_text(df_profile.loc['skills', 'info'])
#### THIS IS ONLY FOR JUPYTER !!!!!!!!!!!!!!!!!!!!
profile_skills = profile_skills.replace('[','').replace(']','').replace("'","").split(', ')

sorted(profile_skills)

['analytical skills',
 'bash',
 'business analysis',
 'business intelligence',
 'business process improvement',
 'continuous improvement',
 'data analysis',
 'data analytics',
 'data modeling',
 'data pipelines',
 'data science',
 'data visualization',
 'engineering',
 'english',
 'español',
 'financial analysis',
 'git',
 'industry knowledge',
 'jupyter',
 'leadership',
 'lean manufacturing',
 'machine learning',
 'microsoft excel',
 'microsoft office',
 'microsoft power bi',
 'microsoft power query',
 'microsoft project',
 'mysql',
 'negotiation',
 'numpy',
 'pandas',
 'power pivot',
 'process engineering',
 'process optimization',
 'programming',
 'project management',
 'python',
 'r',
 'sap',
 'scrum',
 'seaborn',
 'six sigma',
 'sql',
 'statistics',
 'tableau',
 'team management',
 'teamwork',
 'telco',
 'vba',
 'visual basic for applications',
 'web scraping']

In [79]:
df_jobs['PROFILE coincidences'] = \
df_jobs['Job Description'].apply(lambda x:      coincidencias(profile_skills,clean_text(x)))
df_jobs['PROFILE coincidences_list'] = \
df_jobs['Job Description'].apply(lambda x: list_coincidencias(profile_skills,clean_text(x)))

In [80]:
df_jobs.head(2)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,...,Multiple Job Post,Info Language,an coincidences,an coincidences_list,ci coincidences,ci coincidences_list,in coincidences,in coincidences_list,PROFILE coincidences,PROFILE coincidences_list
0,analista datos,Barcelona,2020-04-23 00:40,1712815780,https://www.linkedin.com/jobs/view/1712815780/,Analista Liferay,everis,"Barcelona y alrededores, España",1 week,2020-04-16 00:40,...,AD---,es,0,[],0,[],1,[java ],2,"[git, r]"
1,analista datos,Barcelona,2020-04-23 00:40,1813681738,https://www.linkedin.com/jobs/view/1813681738/,"Analista, programadores/as y manager Salesforce",Experis España,"Barcelona, Catalonia, Spain",2 weeks,2020-04-09 00:40,...,AD---,en,1,[business analyst],1,[business analyst],0,[],2,"[business analysis, r]"


#### checking results >

In [81]:
check = 5

In [82]:
df_jobs.loc[check ,'Job name']

'Innova-tsn: Analista de datos'

In [84]:
dict(df_jobs.loc[check ,'ad coincidences':'PROFILE coincidences_list'])

KeyError: 'ad coincidences'

In [ ]:
clean_text(df_jobs.loc[check ,'Job Description'])

### TOP JOB POST >

In [85]:
top = 5

In [86]:
df_top_jobs = df_jobs\
.drop_duplicates(subset='Current Job Id', keep='first')\
.sort_values(by=['PROFILE coincidences'], ascending=False).head(top)\
[['JOB TITLE', 'Job name','Company name','Company location','Posted date','Employment Type',
  'Job html','Multiple Job Post']]
df_top_jobs

,JOB TITLE,Job name,Company name,Company location,Posted date,Employment Type,Job html,Multiple Job Post
95,ingeniero datos,Data Scientist Internship,Entravision,"Barcelona, ES",3 days,Internship,https://www.linkedin.com/jobs/view/1832936591/,--ID-
83,ingeniero datos,Data Scientist (JVBS),Zurich Insurance Company Ltd,"Barcelona, ES",1 week,Full-time,https://www.linkedin.com/jobs/view/1776734400/,--ID-
10,analista datos,Senior Data Analyst - Barcelona,Flight Centre Travel Group,"Barcelona, ES",3 months,Contract,https://www.linkedin.com/jobs/view/1704442844/,AD---
40,analista datos,Senior Data Analyst,Fcm,"Barcelona, ES",3 weeks,Contract,https://www.linkedin.com/jobs/view/1830200694/,AD---
82,ingeniero datos,Data Scientist (JVBS),Zurich,"Barcelona, ES",1 month,Full-time,https://www.linkedin.com/jobs/view/1779904136/,--ID-


In [87]:
df_top_jobs['Job html'] = df_top_jobs['Job html'].apply(lambda x: f'<a href={x}>link</a>')

In [88]:
df_top_jobs.head(10)

,JOB TITLE,Job name,Company name,Company location,Posted date,Employment Type,Job html,Multiple Job Post
95,ingeniero datos,Data Scientist Internship,Entravision,"Barcelona, ES",3 days,Internship,<a href=https://www.linkedin.com/jobs/view/183...,--ID-
83,ingeniero datos,Data Scientist (JVBS),Zurich Insurance Company Ltd,"Barcelona, ES",1 week,Full-time,<a href=https://www.linkedin.com/jobs/view/177...,--ID-
10,analista datos,Senior Data Analyst - Barcelona,Flight Centre Travel Group,"Barcelona, ES",3 months,Contract,<a href=https://www.linkedin.com/jobs/view/170...,AD---
40,analista datos,Senior Data Analyst,Fcm,"Barcelona, ES",3 weeks,Contract,<a href=https://www.linkedin.com/jobs/view/183...,AD---
82,ingeniero datos,Data Scientist (JVBS),Zurich,"Barcelona, ES",1 month,Full-time,<a href=https://www.linkedin.com/jobs/view/177...,--ID-


In [89]:
## render dataframe as html
html = df_top_jobs.to_html(escape=False)

In [91]:
#write html to file
if platform.system() == 'Linux':
    html_file = f"../data/results/top_{top}_jobs_in_{location}.html"
    text_file = open(html_file, "w")
    text_file.write(html)
    text_file.close()
    #text_file.close()
    webbrowser.open(html_file, new=1)
    
elif platform.system() == 'Windows':
    text_file = open(f"..\\data\\results\\top_{top}_jobs_in_{location}.html", "w")
    text_file.write(html)
    #text_file.close()

#### checking results >

In [ ]:
df_jobs[df_jobs['Current Job Id']==1682047755]

## Using NLP

In [84]:
nlp_en = spacy.load('en_core_web_md')
nlp_es = spacy.load('es_core_news_md')

In [85]:
df_jobs.columns

Index(['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
       'Job html', 'Job name', 'Company name', 'Company location',
       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
       'Job info', 'Job Description', 'JOB TITLE_ID', 'Info Language',
       'da coincidences', 'da coincidences_list', 'PROFILE coincidences',
       'PROFILE coincidences_list', 'Multiple Job Post'],
      dtype='object')

In [86]:
unique_jobs[0].split(" ")[0][0].upper() + unique_jobs[0].split(" ")[1][0].upper()

'DA'

In [87]:
example = "data hola kase"
nuevo_key = job_key_generator(example)
nuevo_key

'DHK'

In [88]:
unique_jobs

['data analyst', 'data scientist', 'data engineer']

In [89]:
unique_languages = list(df_jobs['Info Language'].unique())
unique_languages

['es', 'en']

In [90]:
location = df_jobs['LOCATION'].unique()[0].lower()
location

'madrid'

In [91]:
black_list = \
[location,
 'empresa','work','skill',
'parir','comer','timar',
 'job','industry', 'type','employment', 'full','functions','seniority','level',
 'company','experience','help','time','new','role','use','year','enable',
]

In [92]:
dict_corpus = {}
dict_filtered_corpus = {}
dict_filtered_tokens = {}
dict_word_freq = {}
dict_common_words_100 = {}

dict_filtered_corpus_NN = {}
dict_filtered_tokens_NN = {}
dict_word_freq_NN = {}
dict_common_words_100_NN = {}

In [ ]:
%%time
for job in unique_jobs:
    

    job_key_s = job_key_generator(job)
    
    for lang in unique_languages:
        
        #CORPUS
        filter_job  = df_jobs['JOB TITLE'] == job
        filter_lang = df_jobs['Info Language'] == lang
        corpus = df_jobs['Job Description'][filter_job & filter_lang].str.cat(sep=" || ")
        
        dict_corpus.update( {f'{job_key_s}_{lang}' : corpus} )
        
        
        #TOKENS
        if   lang == 'en':
            tokens = nlp_en(corpus)
            STOP_WORDS = SW_en
        elif lang == 'es':
            tokens = nlp_es(corpus)
            STOP_WORDS = SW_es
        
        filtered_corpus = ""
        filtered_tokens = []
        #NOUNS
        filtered_corpus_NN = ""
        filtered_tokens_NN = []

        for word in tokens:
            lemma = word.lemma_.lower().strip()
            NN    = word.pos_.lower().strip()

            if lemma not in STOP_WORDS and re.search('^[a-z]+$', lemma) and lemma not in black_list and len(lemma)>1:
                    filtered_corpus += lemma + " "
                    filtered_tokens.append(lemma)
                    
                    #NOUNS
                    if word.pos_ == 'NOUN':
                        filtered_corpus_NN += lemma + " "
                        filtered_tokens_NN.append(lemma)
        
        dict_filtered_corpus.update( {f'{job_key_s}_{lang}': filtered_corpus} )
        dict_filtered_tokens.update( {f'{job_key_s}_{lang}': filtered_tokens} )
        
        #NOUNS
        dict_filtered_corpus_NN.update( {f'{job_key_s}_{lang}': filtered_corpus_NN} )
        dict_filtered_tokens_NN.update( {f'{job_key_s}_{lang}': filtered_tokens_NN} )
        
        #100 COMMON WORDS
        word_freq = Counter(filtered_tokens)
        common_words_100 = word_freq.most_common(100)
        dict_word_freq.update({f'{job_key_s}_{lang}': word_freq})
        dict_common_words_100.update({f'{job_key_s}_{lang}': common_words_100})
        
        #NOUNS
        word_freq_NN = Counter(filtered_tokens_NN)
        common_words_100_NN = word_freq_NN.most_common(100) 
        dict_word_freq_NN.update({f'{job_key_s}_{lang}': word_freq_NN})
        dict_common_words_100_NN.update({f'{job_key_s}_{lang}': common_words_100_NN})
        
        if platform.system() == 'Linux':
            with open(f'../data/results/dict_word_freq{job_key_s}_{lang}.json', 'w') as fp:
                json.dump(dict_word_freq, fp)
        elif platform.system() == 'Windows':
            with open(f'..\\data\\results\\dict_word_freq{job_key_s}_{lang}.json', 'w') as fp:
                json.dump(dict_word_freq, fp)

#### checking results >

In [ ]:
job_lang = 'DS_es'

In [ ]:
dict_corpus.keys()

In [ ]:
dict_corpus.get(job_lang)[400:600]

In [ ]:
dict_filtered_tokens.keys()

In [ ]:
dict_filtered_tokens.get(job_lang)[600:610]

In [ ]:
dict_word_freq.keys()

In [ ]:
dict_common_words_100.keys()

In [ ]:
dict_common_words_100.get(job_lang)[0:20]

In [ ]:
dict_common_words_100_NN.get(job_lang)[0:40]

## CLOUD 

In [ ]:
unique_jobs

In [ ]:
unique_languages

In [ ]:
#def cloud_word(unique_jobs):

for j in unique_jobs:
    

    """
    if platform.system() == 'Linux':
        try:
            mask = np.array(Image.open(f"../data/raw/{j}.png"))
        except:
            mask = np.array(Image.open(f"../data/raw/{j}.png"))

    elif platform.system() == 'Windows':
        try:
            mask = np.array(Image.open(f"..\\data\\raw\\{j}.png"))
        except:
            mask = np.array(Image.open(f"..\\data\\raw\\{j}.png"))

    image_colors = ImageColorGenerator(mask)
    """

    job_key_w = job_key_generator(j)
    plot_rows = len(unique_languages)

    fig , axes = plt.subplots(ncols=1, nrows=plot_rows)

    ax = 0
    for lang in unique_languages:
        print(f'{job_key_w}_{lang}')
        plt.axis("off")
        word_corpus = dict_filtered_corpus.get(f'{job_key_w}_{lang}')
        wordcloud = \
        WordCloud(max_font_size=100, max_words=100, background_color="white")\
                .generate(word_corpus)
        #, mask=mask)\
        axes[ax].axis("off")
        axes[ax].set_title(f'{job_key_w}_{lang}')
        axes[ax].imshow(wordcloud, interpolation='bilinear')

        ax+=1
    
    plt.axis("off") 
    fig.get_figure()
    plt.show()


    if platform.system() == 'Linux':
        fig.savefig(f'../data/results/worldcloud_{job_key_w}_{location}.png', quality =95, dpi=300)
        #wordcloud.to_file(f'../data/results/worldcloud_{job_lang}_{location}.png')

    elif platform.system() == 'Windows':
        fig.savefig(f'..\\data\\results\\worldcloud_{job_key_w}_{location}.png', quality =95, dpi=300)
        #wordcloud.to_file(f'..\\data\\results\\worldcloud_{job_lang}_{location}.png')
        
    

In [ ]:
#cloud_word(unique_jobs)

In [ ]:
# Display the generated image:
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

## Using ML

***
# TESTING

### Counting frequent words

In [ ]:
c = re.findall('\{(.*)\}',str(dict_common_words.get(job_lang)))[0].split(',')
c

In [ ]:
word_freq = Counter(dict_filtered_tokens.get('DA_en'))
type(word_freq)

In [ ]:
common_words = word_freq.most_common
type(common_words)

In [ ]:
common_words

### masks in CLOUD

In [ ]:
if platform.system() == 'Linux':
    mask = np.array(Image.open("../data/raw/plot.png"))
    
elif platform.system() == 'Windows':
    mask = np.array(Image.open("../data/raw/monitor.png"))

    
image_colors = ImageColorGenerator(mask)

In [ ]:
wordcloud = WordCloud(max_font_size=100, max_words=100, mode="RGBA", background_color="white", mask=mask)\
            .generate(dict_filtered_corpus.get(job_lang))

In [ ]:
plt.figure(figsize=[7,7])
plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation='bilinear')
plt.axis("off")
plt.show()